In [1]:
from importlib.metadata import version
import os

try:
    import xradio

    print("XRADIO version", version("xradio"), "already installed.")
except ImportError as e:
    print(e)
    print("Installing XRADIO")

    os.system("pip install xradio")

    import xradio

    print("xradio version", version("xradio"), " installed.")

# These are the three API functions that are currently supported
from xradio.image import load_image, read_image, write_image

XRADIO version 0.0.25 already installed.


## Download dataset

fitsidi from https://almascience.eso.org/almadata/ec/eht/2016.1.01114.V/group.uid___A001_X87c_X245.ec_jlgomez.e17a10-7-hi-oj287-3C279-fits.tgz

```python
importfitsidi('E17A10.0.bin0000.source0000.FITS',vis='E17A10.0.bin0000.source0000.ms')
mstransform(vis='E17A10.0.bin0000.source0000.ms',outputvis='ngEHT_E17A10.0.bin0000.source0000_split_lsrk.ms',spw='8:55~61,29:20~27', regridms=True,outframe='lsrk',datacolumn='all')
```

In [2]:
import graphviper

graphviper.utils.data.download(file="ngEHT_E17A10.0.bin0000.source0000_split.ms")

[2024-04-11 10:36:57,202]     INFO  graphviper:  Couldn't find file metadata locally in /Users/jsteeb/Dropbox/graphviper/src/graphviper/utils/data/.dropbox/file.download.json, checking remote API ... 


ngEHT_E17A10.0.bin0000.source0000_split.ms.zip:   0%|          | 0.00/10.6M [00:00<?, ?iB/s]

[2024-04-11 10:36:59,484]     INFO  graphviper:  Extracting file: ngEHT_E17A10.0.bin0000.source0000_split.ms.zip ... 


## Convert to Processing Set

In [3]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

ms_file = "ngEHT_E17A10.0.bin0000.source0000_split.ms"
partition_scheme = "ddi_intent_field"
# partition_scheme="ddi_state"

main_chunksize = {"frequency": 1, "time": 20}  # baseline, polarization
outfile = "ngEHT_E17A10.0.bin0000.source0000_split_lsrk.vis.zarr"
convert_msv2_to_processing_set(
    in_file=ms_file,
    out_file=outfile,
    partition_scheme=partition_scheme,
    parallel=False,
    overwrite=True,
    main_chunksize=main_chunksize,
)

## Processing Set

In [4]:
from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set(ps_store=outfile)
ps.summary()

,name,ddi,intent,field_id,field_name,start_frequency,end_frequency,shape,field_coords
0,ngEHT_E17A10.0.bin0000.source0000_split_lsrk_d...,0,None,0,3C279,2.286593e+11,2.286623e+11,"(2250, 20, 7, 4)","[fk5, 12h56m11.166567s, -5d47m21.52481s]"
1,ngEHT_E17A10.0.bin0000.source0000_split_lsrk_d...,1,None,0,3C279,2.298723e+11,2.298758e+11,"(2250, 20, 8, 4)","[fk5, 12h56m11.166567s, -5d47m21.52481s]"


In [5]:
ps.keys()

dict_keys(['ngEHT_E17A10.0.bin0000.source0000_split_lsrk_ddi_0_intent_None_field_id_0', 'ngEHT_E17A10.0.bin0000.source0000_split_lsrk_ddi_1_intent_None_field_id_0'])

In [6]:
ps["ngEHT_E17A10.0.bin0000.source0000_split_lsrk_ddi_0_intent_None_field_id_0"]

<xarray.Dataset> Size: 18MB
Dimensions:                     (time: 2250, baseline_id: 20, frequency: 7,
                                 polarization: 4, uvw_label: 3)
Coordinates:
    baseline_antenna1_id        (baseline_id) int32 80B dask.array<chunksize=(20,), meta=np.ndarray>
    baseline_antenna2_id        (baseline_id) int32 80B dask.array<chunksize=(20,), meta=np.ndarray>
  * baseline_id                 (baseline_id) int64 160B 0 1 2 3 ... 16 17 18 19
  * frequency                   (frequency) float64 56B 2.287e+11 ... 2.287e+11
  * polarization                (polarization) <U2 32B 'RR' 'RL' 'LR' 'LL'
  * time                        (time) float64 18kB 1.492e+09 ... 1.492e+09
  * uvw_label                   (uvw_label) <U1 12B 'u' 'v' 'w'
Data variables:
    EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 360kB dask.array<chunksize=(20, 20), meta=np.ndarray>
    FLAG                        (time, baseline_id, frequency, polarization) bool 1MB dask.array<chunksize=(20, 20, 1, 4), meta=np.ndarray>
    TIME_CENTROID               (time, baseline_id) float64 360kB dask.array<chunksize=(20, 20), meta=np.ndarray>
    UVW                         (time, baseline_id, uvw_label) float64 1MB dask.array<chunksize=(20, 20, 3), meta=np.ndarray>
    VISIBILITY                  (time, baseline_id, frequency, polarization) complex64 10MB dask.array<chunksize=(20, 20, 1, 4), meta=np.ndarray>
    WEIGHT                      (time, baseline_id, frequency, polarization) float32 5MB dask.array<chunksize=(20, 20, 1, 4), meta=np.ndarray>
Attributes:
    data_groups:   {'base': {'flag': 'FLAG', 'uvw': 'UVW', 'visibility': 'VIS...
    ddi:           0
    intent:        None
    antenna_xds:   <xarray.Dataset> Size: 1kB\nDimensions:        (antenna_id...
    pointing_xds:  <xarray.Dataset> Size: 0B\nDimensions:  ()\nData variables...

In [7]:
ps["ngEHT_E17A10.0.bin0000.source0000_split_lsrk_ddi_0_intent_None_field_id_0"].attrs[
    "antenna_xds"
]

<xarray.Dataset> Size: 1kB
Dimensions:        (antenna_id: 6, xyz_label: 3)
Coordinates:
  * antenna_id     (antenna_id) int64 48B 0 1 2 3 4 5
    mount          (antenna_id) <U16 384B dask.array<chunksize=(6,), meta=np.ndarray>
    name           (antenna_id) <U2 48B dask.array<chunksize=(6,), meta=np.ndarray>
    station        (antenna_id) <U2 48B dask.array<chunksize=(6,), meta=np.ndarray>
    type           (antenna_id) <U12 288B dask.array<chunksize=(6,), meta=np.ndarray>
  * xyz_label      (xyz_label) <U1 12B 'x' 'y' 'z'
Data variables:
    DISH_DIAMETER  (antenna_id) float64 48B dask.array<chunksize=(6,), meta=np.ndarray>
    FEED_OFFSET    (antenna_id, xyz_label) float64 144B dask.array<chunksize=(6, 3), meta=np.ndarray>
    POSITION       (antenna_id, xyz_label) float64 144B dask.array<chunksize=(6, 3), meta=np.ndarray>